## FranceConnect

On essaye dans un premier temps de travailler avec selenium

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

driver = webdriver.Chrome()

url = 'https://franceconnect.gouv.fr/faq'
driver.get(url)

wait = WebDriverWait(driver, 10)

faq_entries = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//ul[contains(@class, "collapsible")]/li')))

questions = []
answers = []

for entry in faq_entries:
    header = entry.find_element(By.XPATH, './/div[contains(@class, "collapsible-header")]')
    driver.execute_script("arguments[0].scrollIntoView(true);", header)
    driver.execute_script("arguments[0].click();", header)

    # On attend que le contenu de la question soit visible
    wait.until(EC.visibility_of(entry.find_element(By.XPATH, './/div[contains(@class, "collapsible-body")]')))
    
    question = header.text.strip()
    answer = entry.find_element(By.XPATH, './/div[contains(@class, "collapsible-body")]').text.strip()

    questions.append(question)
    answers.append(answer)

faq_data = pd.DataFrame({
    'Question': questions,
    'Réponse': answers,
    'Source': 'https://franceconnect.gouv.fr/faq'
    
})


faq_data


In [118]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import pandas as pd

driver = webdriver.Chrome()
driver.get('https://franceconnect.gouv.fr/faq')
wait = WebDriverWait(driver, 10)

In [160]:
## Ouvre chaque première question de chaque partie correctement, maintenant
## on doit ouvrir les autres questions de chaque ensemble
for i in range(1,31):
    try :
        driver.find_elements(By.XPATH,f"/html/body/main/div/div[1]/div[{i}]/div/ul/li/div[1]")[0].click()
        for j in range(2,10):
            try :
                driver.find_elements(By.XPATH,f"/html/body/main/div/div[1]/div[{i}]/div/ul/li[{j}]/div[1]")[0].click()
            except:
                pass
    except:
        pass
    # try :
    #     driver.find_elements(By.XPATH,f"/html/body/main/div/div[1]/div[{i}]/div/ul/li/div[1]")[0].click()
    #     # try :
    #     #     for j in range(1,10):
    #     #         driver.find_elements(By.XPATH,f"/html/body/main/div/div[1]/div[{i}]/div/ul/li[{j}]/div[1]")[0].click()
    #     # except:
    #     #     pass
    # except:
    #     pass

Avec selenium, il est compliqué de scapper les FAQ Dynamiques puisque le clique est nécessaire. Hors on remarque très rapideement que le texte est disponible dans la page source sans même cliquer au préalable. Nous allons donc utiliser BS4

In [284]:
import requests
from bs4 import BeautifulSoup

url = "https://franceconnect.gouv.fr/faq"
response = requests.get(url)
content = response.content

soup = BeautifulSoup(content, "html.parser")

question_elements = soup.find_all("div", class_="collapsible-header")
answer_elements = soup.find_all("div", class_="collapsible-body")


question_list = []
answer_list = []

for question, answer in zip(question_elements, answer_elements):
    question_text = question.text.strip()
    answer_text = answer.text.strip()
    question_list.append(question_text)
    answer_list.append(answer_text)

faq_data = pd.DataFrame({
    'Question': question_list,
    'Réponse': answer_list,
    'Source': 'franceconnect.gouv',
    'Langue': 'français'
})

In [218]:
faq_data

,Question,Réponse,Source,Langue
0,Fraude à Mon Compte Formation,Des appels frauduleux sont actuellement en cou...,franceconnect.gouv,français
1,Phishing / Hameçonnage,Une campagne de phishing est actuellement en c...,franceconnect.gouv,français
2,Comment utiliser FranceConnect ?,Vous avez déjà un compte auprès d’Impots.gouv....,franceconnect.gouv,français
3,Eligibilité à FranceConnect ?,"Pour utiliser FranceConnect, vous devez:\n\n\n...",franceconnect.gouv,français
4,Pourquoi je reçois un mail m’informant que je ...,Systématiquement pour chaque connexion via Fra...,franceconnect.gouv,français
5,Je veux modifier l’adresse email sur laquelle ...,Il vous faut changer cette adresse sur le comp...,franceconnect.gouv,français
6,Je tente une connexion FranceConnect et reçois...,Vous ne pouvez pas vous connecter via FranceCo...,franceconnect.gouv,français
7,Je tente une connexion FranceConnect et je reç...,Vous ne pouvez pas vous connecter via FranceCo...,franceconnect.gouv,français
8,Je tente une connexion FranceConnect et je reç...,Cette erreur de connexion indique que la sessi...,franceconnect.gouv,français
9,Je ne reçois pas d’email lorsque je clique sur...,Vous avez changé de messagerie sans mettre à j...,franceconnect.gouv,français


# Etudiant 

In [285]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

urls = [
    "https://www.etudiant.gouv.fr/fr/faq-cvec-2402",
    "https://www.etudiant.gouv.fr/fr/faq-la-cesure-comment-ca-marche-1453",
    "https://www.etudiant.gouv.fr/fr/faq-stages-le-point-sur-vos-droits-589"
]

faq_etudiant = pd.DataFrame(columns=["Question", "Réponse", "Source", "Langue"])

for url in urls:
    response = requests.get(url)
    content = response.content

    soup = BeautifulSoup(content, "html.parser")
    terms = soup.find_all("div", class_="term")

    for term in terms:
        question = term.find("div", class_="question-answer-wrapper").find("button").text.strip()
        answer = term.find("div", class_="answer").text.strip()

        if not faq_etudiant[faq_etudiant['Question'] == question].empty:
            continue  

        new_row = pd.DataFrame({
            "Question": [question],
            "Réponse": [answer],
            "Source": [url],
            "Langue": ["Français"]
        })
        faq_etudiant = pd.concat([faq_etudiant, new_row], ignore_index=True)

faq_etudiant


,Question,Réponse,Source,Langue
0,"C'est quoi, concrètement, la CVEC ?",La CVEC est la Contribution de vie étudiante e...,https://www.etudiant.gouv.fr/fr/faq-cvec-2402,Français
1,Quel est le montant de la CVEC ?,Le montant de la CVEC est de 100 € pour l'anné...,https://www.etudiant.gouv.fr/fr/faq-cvec-2402,Français
2,Qui est concerné par la CVEC ?,Les étudiants inscrits en formation initiale d...,https://www.etudiant.gouv.fr/fr/faq-cvec-2402,Français
3,Quand dois-je m'acquitter de la CVEC ?,Vous devez vous acquitter de votre CVEC avant ...,https://www.etudiant.gouv.fr/fr/faq-cvec-2402,Français
4,Comment m'acquitter de la CVEC ?,Pour vous acquitter de la CVEC et obtenir votr...,https://www.etudiant.gouv.fr/fr/faq-cvec-2402,Français
...,...,...,...,...
85,Ai-je d'autres droits en stage ?,Vous bénéficiez des mêmes droits que les salar...,https://www.etudiant.gouv.fr/fr/faq-stages-le-...,Français
86,Quelles sont les obligations du stagiaire et l...,Vous êtes tenu de respecter ce qui est prévu d...,https://www.etudiant.gouv.fr/fr/faq-stages-le-...,Français
87,Quelles règles pour les étudiants internationa...,"Étudiants issus de l’Union européenne, d’un de...",https://www.etudiant.gouv.fr/fr/faq-stages-le-...,Français
88,Comment me protéger lors de mon stage ?,"La couverture maladie\nSauf cas particuliers, ...",https://www.etudiant.gouv.fr/fr/faq-stages-le-...,Français


# Ryanair

Nous avons tenté de scrapper ce site, il fallait trouver des alternatives mais nous n'avons pas réussi

In [282]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
import time

driver = webdriver.Chrome()
driver.get("https://help.ryanair.com/hc/fr-fr/categories/12489248662673-R%C3%A9servations-aupr%C3%A8s-d-une-agence-de-voyages")
wait = WebDriverWait(driver, 10)

faq_ryanair = pd.DataFrame(columns=["Question", "Réponse", "Source", "Langue"])

blocs = driver.find_elements(By.CSS_SELECTOR, 'a[href*="articles"]')
for bloc in blocs:
    try:
        bloc.click() 
        time.sleep(1)  
    except:
        continue

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    articles = soup.find_all('div', class_='article-toggle')

    for article in articles:
        question = article.find('a').text.strip() if article.find('a') else 'No question'
        answer = article.find('div', class_='content').text.strip() if article.find('div', class_='content') else 'No answer'
        
        if not faq_ryanair[faq_ryanair['Question'] == question].empty:
            continue
        new_row = {'Question': question, 'Réponse': answer, 'Source': url, 'Langue': 'Français'}
        faq_ryanair = faq_ryanair.append(new_row, ignore_index=True)

df = pd.DataFrame(faq_ryanair)


# Fermer le navigateur
driver.quit()

# Afficher les résultats
print(df)


Empty DataFrame
Columns: [Question, Réponse, Source, Langue]
Index: []


# Data.gouv

In [287]:
all_list = []

for k in range(1, 21):
    current_lis = []
    
    for i in range(1, 21):
        try:
            ns = driver.find_element(By.XPATH, f'/html/body/div[3]/main/div[3]/div/div[2]/ul[{k}]/li[{i}]/h5/button')
            current_lis.append(ns.text)
        except :
            pass
    
    all_list.append(current_lis)

flat_lis = []
for sublist in all_list:
    for item in sublist:
        flat_lis.append(item)

import pandas as pd
flat_list = []
for sublist in all_list:
    for item in sublist:
        flat_list.append(item)
ydf = pd.DataFrame(
    {'Questions': flat_lis,
     'Answers': flat_list,
     'Source' : "data.gouv",
     "Langue":"Français"
    })
ydf

,Questions,Answers,Source,Langue


In [288]:
faq_total = pd.concat([faq_data, faq_etudiant,ydf], ignore_index=True)
faq_total

,Question,Réponse,Source,Langue,Questions,Answers
0,Fraude à Mon Compte Formation,Des appels frauduleux sont actuellement en cou...,franceconnect.gouv,français,NaN,NaN
1,Phishing / Hameçonnage,Une campagne de phishing est actuellement en c...,franceconnect.gouv,français,NaN,NaN
2,Comment utiliser FranceConnect ?,Vous avez déjà un compte auprès d’Impots.gouv....,franceconnect.gouv,français,NaN,NaN
3,Eligibilité à FranceConnect ?,"Pour utiliser FranceConnect, vous devez:\n\n\n...",franceconnect.gouv,français,NaN,NaN
4,Pourquoi je reçois un mail m’informant que je ...,Systématiquement pour chaque connexion via Fra...,franceconnect.gouv,français,NaN,NaN
...,...,...,...,...,...,...
125,Ai-je d'autres droits en stage ?,Vous bénéficiez des mêmes droits que les salar...,https://www.etudiant.gouv.fr/fr/faq-stages-le-...,Français,NaN,NaN
126,Quelles sont les obligations du stagiaire et l...,Vous êtes tenu de respecter ce qui est prévu d...,https://www.etudiant.gouv.fr/fr/faq-stages-le-...,Français,NaN,NaN
127,Quelles règles pour les étudiants internationa...,"Étudiants issus de l’Union européenne, d’un de...",https://www.etudiant.gouv.fr/fr/faq-stages-le-...,Français,NaN,NaN
128,Comment me protéger lors de mon stage ?,"La couverture maladie\nSauf cas particuliers, ...",https://www.etudiant.gouv.fr/fr/faq-stages-le-...,Français,NaN,NaN
